In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [2]:
def pre_lbp(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rows, cols = gray.shape[0], gray.shape[1]
    pre_lbp = np.zeros((rows, cols), dtype=np.uint8)  
    return gray, rows, cols, pre_lbp


def calculate_lbp_clockwise(image):
    gray, rows, cols, lbp_clockwise = pre_lbp(image)
    for i in range(1, rows - 1):  # Hindari satu piksel di tepi atas dan bawah
        for j in range(1, cols - 1):  # Hindari satu piksel di tepi kiri dan kanan
            center = gray[i, j]
            code = 0
            
            # Tetangga dalam urutan searah jarum jam mulai dari 3 o'clock
            neighbors = [gray[i - 1, j], gray[i - 1, j + 1], gray[i, j + 1],
                         gray[i + 1, j + 1], gray[i + 1, j], gray[i + 1, j - 1],
                         gray[i, j - 1], gray[i - 1, j - 1]]
            
            # Hitung perbedaan antara piksel tetangga dan piksel pusat
            for k in range(8):
                if neighbors[k] >= center:
                    code |= (1 << k)
            
            lbp_clockwise[i - 1, j - 1] = code
    return lbp_clockwise

def calculate_lbp_anticlockwise(image):
    gray, rows, cols, lbp_anticlockwise = pre_lbp(image)
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            center = gray[i, j]
            code = 0
            
            # Tetangga dalam urutan berlawanan arah jarum jam mulai dari 9 o'clock
            neighbors = [gray[i - 1, j - 1], gray[i - 1, j], gray[i - 1, j + 1],
                         gray[i, j + 1], gray[i + 1, j + 1], gray[i + 1, j], gray[i + 1, j - 1],
                         gray[i, j - 1]]
            
            # Hitung perbedaan antara piksel tetangga dan piksel pusat
            for k in range(8):
                if neighbors[k] >= center:
                    code |= (1 << k)
            
            lbp_anticlockwise[i - 1, j - 1] = code
    
    return lbp_anticlockwise

    
def calculate_ncdblbp_clockwise(image):
    gray, rows, cols, ncdlbp_clockwise = pre_lbp(image)
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            center = gray[i, j]
            code = 0
            
            # Tetangga dalam urutan searah jarum jam mulai dari 3 o'clock
            neighbors = [gray[i - 1, j], gray[i - 1, j + 1], gray[i, j + 1],
                         gray[i + 1, j + 1], gray[i + 1, j], gray[i + 1, j - 1],
                         gray[i, j - 1], gray[i - 1, j - 1]]
            
            # Hitung perbedaan antara piksel tetangga dan piksel pusat
            for k in range(8):
                if neighbors[k] >= center:
                    code |= (1 << k)
            
            ncdlbp_clockwise[i, j] = code
    
    return ncdlbp_clockwise


def calculate_ncdblbp_anticlockwise(image):
    gray, rows, cols, ncdlbp_anticlockwise = pre_lbp(image)
    
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            center = gray[i, j]
            code = 0
            
            # Tetangga dalam urutan berlawanan arah jarum jam mulai dari 9 o'clock
            neighbors = [gray[i - 1, j], gray[i - 1, j - 1], gray[i, j - 1],
                         gray[i + 1, j - 1], gray[i + 1, j], gray[i + 1, j + 1],
                         gray[i, j + 1], gray[i - 1, j + 1]]
            
            # Hitung perbedaan antara piksel tetangga dan piksel pusat
            for k in range(8):
                if neighbors[k] >= center:
                    code |= (1 << k)
            
            ncdlbp_anticlockwise[i, j] = code
    
    return ncdlbp_anticlockwise


In [12]:
list_root_dir = ['dataset-resize-16', 'dataset-resize-32', 'dataset-resize-64', 'dataset-resize-128']

In [13]:
for root_directory in list_root_dir:
    # Tentukan direktori awal yang ingin Anda iterasi
    # root_directory = 'dataset-resize'

    data = []

    # Iterasi semua folder, subfolder, dan file dalam direktori
    for root, directories, files in os.walk(root_directory):
        # Cetak nama folder saat ini
        folder_name = os.path.basename(root)

        # Iterasi semua subfolder dalam folder saat ini (root)
        for directory in directories:
            subfolder_path = os.path.join(root, directory)
            print(directory)
            # Iterasi semua file dalam subfolder saat ini
            for file in os.listdir(subfolder_path):
                file_path = os.path.join(subfolder_path, file)
                image_path = file_path
                image = cv2.imread(image_path)
                
                temp_lbp = calculate_ncdblbp_anticlockwise(image)
                temp_lbp = temp_lbp.flatten()
                temp_lbp = temp_lbp.astype(object)
                temp_lbp = np.insert(temp_lbp, 0, directory)
                data.append(temp_lbp)
                # print(f'{directory}/{file}')

    df = pd.DataFrame(data)
    df.head()
    
    df.to_csv(f'{root_directory}.csv')

Angry
Happy
Other
Sad
Angry
Happy
Other
Sad
Angry
Happy
Other
Sad
Angry
Happy
Other
Sad


In [9]:
df = pd.DataFrame(data)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,39991,39992,39993,39994,39995,39996,39997,39998,39999,40000
0,Angry,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Angry,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Angry,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Angry,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Angry,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


0        0
1        0
2        0
3        0
4        0
        ..
39996    0
39997    0
39998    0
39999    0
40000    0
Length: 40001, dtype: int64

In [5]:
# # Tentukan threshold (50% dari panjang DataFrame)
# threshold = len(df) * 0.7

# # Menghapus kolom di mana lebih dari 50% nilai adalah NaN
# df = df.dropna(axis=1, thresh=threshold)
# df.head()

In [6]:
# df = df.fillna(0)
# df

In [7]:
# df[0].value_counts()

In [8]:
# df.to_csv('dataset-ncdblbpac.csv')